In [1]:
import cv2
import numpy as np
import random
import tensorflow as tf

In [2]:
# Basic parameters
image_width = 64
image_height = 64
num_images = 10000
line_min_length = 10

# Lists of train, test, validation images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []
val_images = []
val_labels = []


# Generating images and labels
for i in range(num_images):
    # White background
    image = np.ones((image_height, image_width, 1), dtype=np.uint8) * 255
    
    # Generating coordinates
    x_start = random.randint(line_min_length + 1, image_width - line_min_length -1)
    y_start = random.randint(line_min_length + 1, image_height - line_min_length -1)
    x_end = random.choice([random.randint(0, x_start - line_min_length), random.randint(x_start + line_min_length, image_width-1)])
    y_end = random.choice([random.randint(0, y_start - line_min_length), random.randint(y_start + line_min_length, image_width-1)])
    
    # Drawing a line
    image = cv2.line(image, (x_start, y_start), (x_end, y_end), (0, 0, 0), 1)
    
    # Processing the data for CNN and standarization
    image = image.astype('float32') / 255.0
    
    
    # Spliting the dataset
    if i < 0.8 * num_images:
        train_images.append(image)
        train_labels.append([x_start, y_start, x_end, y_end])
    elif i < 0.9 * num_images:
        val_images.append(image)
        val_labels.append([x_start, y_start, x_end, y_end])
    else:
        test_images.append(image)
        test_labels.append([x_start, y_start, x_end, y_end])
        
    
# Conversion to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [3]:
# Creating the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)  
])


In [4]:
# Compiling the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

# Model training
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))


# Evaluation of the model using test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
test_accuracy_percent = test_accuracy *100
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy_percent:.2f}%')

Epoch 1/10
250/250 [==============================] - 54s 213ms/step - loss: 237.2502 - accuracy: 0.5562 - val_loss: 77.9733 - val_accuracy: 0.7840
Epoch 2/10
250/250 [==============================] - 52s 209ms/step - loss: 65.4510 - accuracy: 0.7844 - val_loss: 51.1140 - val_accuracy: 0.8100
Epoch 3/10
250/250 [==============================] - 50s 200ms/step - loss: 46.2727 - accuracy: 0.8117 - val_loss: 37.3528 - val_accuracy: 0.8470
Epoch 4/10
250/250 [==============================] - 54s 218ms/step - loss: 35.3596 - accuracy: 0.8424 - val_loss: 32.6392 - val_accuracy: 0.8560
Epoch 5/10
250/250 [==============================] - 58s 230ms/step - loss: 31.3621 - accuracy: 0.8569 - val_loss: 30.1381 - val_accuracy: 0.8570
Epoch 6/10
250/250 [==============================] - 57s 226ms/step - loss: 29.5961 - accuracy: 0.8608 - val_loss: 29.8843 - val_accuracy: 0.8510
Epoch 7/10
250/250 [==============================] - 51s 205ms/step - loss: 28.1814 - accuracy: 0.8674 - val_loss: 2

In [5]:
# Model predictions
# Picking random image
random_index = np.random.randint(0, len(test_images))
img = test_images[random_index]
img = np.expand_dims(img, axis=0)

# Making predictions
predictions = model.predict(img)
x_start_pred, y_start_pred, x_end_pred, y_end_pred = predictions[0]
x_start_actual, y_start_actual, x_end_actual, y_end_actual = test_labels[random_index]

# Comparing predictions to actual coordinates
print(f"Actual coordinates for image number {random_index}:")
print("Start coordinates")
print("x_start:", x_start_actual)
print("y_start:", y_start_actual)
print("End coordinates")
print("x_end:", x_end_actual)
print("y_end:", y_end_actual)

print('\n')

print(f"Predictions for image number {random_index}:")
print("Start coordinates")
print("x_start:", round(x_start_pred))
print("y_start:", round(y_start_pred))
print("End coordinates")
print("x_end:", round(x_end_pred))
print("y_end:", round(y_end_pred))

Actual coordinates for image number 952:
Start coordinates
x_start: 23
y_start: 45
End coordinates
x_end: 9
y_end: 23


Predictions for image number 952:
Start coordinates
x_start: 23
y_start: 45
End coordinates
x_end: 7
y_end: 25
